In [ ]:
# importing necessary libraries
import numpy as np 
import pandas as pd 
import seaborn as sns
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from nltk.corpus import stopwords
from nltk.util import ngrams

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# loading our dataset into pandas variable df
df = pd.read_csv('/content/Data/ResultFile (2).csv')
myCopy = df.copy()

df.shape

(8245, 12)

In [ ]:
# dropping tweet_id column since we are now concerned with the tweets' text
df.drop(['tweet_id'], axis = 1, inplace= True)

In [ ]:
from nltk.stem.porter import *
plt.style.use('seaborn')
import plotly.express as px
from plotly import graph_objs as go
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
for index, row in df.iterrows():
    tweet = row.text
    tweet = re.sub(r"https\S+", "", tweet) #remove links
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    
    df['text'].iloc[index:index+1] = tweet

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#displaying tweets after above cleaning
df['text'].head()

0    Domestic Violence Awareness Hasn’t Caught Up W...
1                               Mother Nature’s MeToo 
2    It is 'high time' MeToo named and shamed men i...
3    “There's this idea that persistence involves s...
4                Meredith’s MeToo moment on TheOffice 
Name: text, dtype: object

In [ ]:
# Helper function to remove unwanted patterns
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

# Remove Twitter handles from the data 
df['text'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")


# Remove all words below 2 characters
df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# Remove punctuations, numbers, and special characters
df['text'] = df['text'].str.replace("[^a-zA-Z#]", " ")

df1=df.copy();

# Tokenize the tweets
tokenized_tweet = df['text'].apply(lambda x: x.split())

df['text_tokenised'] = tokenized_tweet

In [ ]:
df['text_tokenised']

0       [Domestic, Violence, Awareness, Hasn, t, Caugh...
1                              [Mother, Nature, s, MeToo]
2       [high, time, MeToo, named, and, shamed, men, m...
3       [There, s, this, idea, that, persistence, invo...
4                 [Meredith, s, MeToo, moment, TheOffice]
                              ...                        
8240      [Metoo, campaign, and, what, labour, laws, say]
8241      [Metoo, campaign, and, what, labour, laws, say]
8242    [Insightful, peace, Hala, Al, Karib, the, dang...
8243    [Olivia, Munn, Opens, About, Predator, Scandal...
8244    [Megan, Fox, feels, excluded, from, the, MeToo...
Name: text_tokenised, Length: 8245, dtype: object

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
from nltk import pos_tag
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))

stops

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
documents = [clean_review(document) for document in df['text_tokenised']]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
data=[]
temp=[]
for x in documents:
  for item in x:
    temp.append(item)

data.append(temp)

In [ ]:
data

[['domestic',
  'violence',
  'awareness',
  'caught',
  'metoo',
  'mother',
  'nature',
  'metoo',
  'high',
  'time',
  'metoo',
  'name',
  'sham',
  'men',
  'medium',
  'advertising',
  'across',
  'india',
  'mumbrella',
  'asia',
  'idea',
  'persistence',
  'involves',
  'systematically',
  'ignore',
  'signal',
  'queue',
  'admit',
  'oth',
  'meredith',
  'metoo',
  'moment',
  'theoffice',
  'interesting',
  'analysis',
  'change',
  'perception',
  'metoo',
  'movement',
  'year',
  'among',
  'amer',
  'shock',
  'hey',
  'old',
  'bill',
  'clear',
  'amp',
  'every',
  'single',
  'imwithher',
  'metoo',
  'moron',
  'swallow',
  'huge',
  'pile',
  'sh',
  'wonder',
  'else',
  'think',
  'lust',
  'love',
  'metoo',
  'bet',
  'saw',
  'come',
  'put',
  'bag',
  'purpose',
  'obviously',
  'day',
  'million',
  'tweet',
  'one',
  'powerful',
  'movement',
  'today',
  'mark',
  'one',
  'year',
  'since',
  'metoo',
  'go',
  'viral',
  'sexual',
  'amp',
  'gender

In [ ]:
import gensim
from gensim.models.word2vec import Word2Vec
LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [ ]:
model_w2v = gensim.models.Word2Vec(
            data,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=5, # Ignores all words with total frequency lower than 5.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 30, # no.of cores
            seed = 34) 

model_w2v.train(data, total_examples= len(df['text']), epochs=20)

(200000, 1430260)

In [ ]:
w1 = "women"
model_w2v.wv.most_similar (positive=w1)

[('conservative', 0.8954508304595947),
 ('victimized', 0.8746390342712402),
 ('nomination', 0.8689011335372925),
 ('rights', 0.867294192314148),
 ('step', 0.8459270596504211),
 ('finally', 0.8402274250984192),
 ('underscored', 0.8384209275245667),
 ('march', 0.8352357149124146),
 ('coming', 0.8349869251251221),
 ('bluewave', 0.8341922760009766)]

In [ ]:
embeddings_index = {}
for w in model_w2v.wv.vocab.keys():
    embeddings_index[w] = model_w2v.wv[w]

In [ ]:
X=df['text']
X

0       Domestic Violence Awareness Hasn t Caught With...
1                                   Mother Nature s MeToo
2        high time  MeToo named and shamed men media a...
3        There s this idea that persistence involves s...
4                       Meredith s MeToo moment TheOffice
                              ...                        
8240              Metoo campaign and what labour laws say
8241              Metoo campaign and what labour laws say
8242    Insightful peace Hala Al Karib the dangers the...
8243    Olivia Munn Opens About  Predator  Scandal and...
8244    Megan Fox feels excluded from the MeToo moveme...
Name: text, Length: 8245, dtype: object

In [ ]:
Y=df[['relevance','directed_hate','generalised_hate','sarcasm','allegation','justification','refutation','Support','oppose']]
Y

,relevance,directed_hate,generalised_hate,sarcasm,allegation,justification,refutation,Support,oppose
0,1,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1,0
3,1,0,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
8240,0,0,0,0,0,0,0,0,0
8241,1,0,0,0,0,0,0,1,0
8242,1,0,0,1,0,0,0,0,0
8243,1,0,0,0,0,0,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=0)
y_train

,relevance,directed_hate,generalised_hate,sarcasm,allegation,justification,refutation,Support,oppose
1042,1,0,0,0,0,0,0,0,0
1116,1,1,0,0,1,0,0,1,0
2211,1,0,0,0,1,0,0,0,1
7132,1,0,0,0,0,0,0,1,0
2404,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
4373,1,0,0,0,0,0,0,1,0
7891,1,1,0,0,1,0,0,1,0
4859,1,0,0,0,0,0,0,0,1
3264,0,0,0,0,0,0,0,0,0


In [ ]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [ ]:
trydata=[]
for d in documents:
  mystr = listToString(d)
  trydata.append(mystr)
trydata

['domestic violence awareness caught metoo',
 'mother nature metoo',
 'high time metoo name sham men medium advertising across india mumbrella asia',
 'idea persistence involves systematically ignore signal queue admit oth',
 'meredith metoo moment theoffice',
 'interesting analysis change perception metoo movement year among amer',
 'shock hey old bill clear amp every single imwithher metoo moron swallow huge pile sh',
 'wonder else think lust love metoo',
 'bet saw come put bag purpose obviously',
 'day million tweet one powerful movement today mark one year since metoo go viral sexual amp gender ba',
 'nsw police officer mitchell willey could kick force admit groped woman defqon dan',
 'poor kid uspolitics auspol ukpol metoo timesup',
 'nsw police officer mitchell willey could kick force admit groped woman defqon dan',
 'metoo literally',
 'grot familiar type individual meet chloeingramm chloe ingram',
 'day million tweet one powerful movement today mark one year since metoo go vira

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(XTrain['commentText'])
sequences = tokenizer.texts_to_sequences(XTrain['commentText'])

In [ ]:
documents[10]

['nsw',
 'police',
 'officer',
 'mitchell',
 'willey',
 'could',
 'kick',
 'force',
 'admit',
 'groped',
 'woman',
 'defqon',
 'dan']

In [ ]:
training_documents = documents[0:6200]
testing_documents = documents[6200:]

In [ ]:
all_words = []
for doc in training_documents:
    all_words += doc
len(all_words)

54165

In [ ]:
documents_corrected = [" ".join(x) for x in documents]

# **TFIDF FEATURE EXTRACTION**

In [ ]:
##Tfidf feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=3000, min_df=5, max_df=0.7)  
X = tfidfconverter.fit_transform(documents_corrected).toarray()  


In [ ]:
from sklearn.model_selection import train_test_split  
## splitting for relevance
X_train_relevance, X_test_relevance, y_train_relevance, y_test_relevance = train_test_split(X,df['relevance'] , test_size=0.3, random_state=0)

## splitting for directed_hate
X_train_dh, X_test_dh, y_train_dh, y_test_dh = train_test_split(X,df['directed_hate'] , test_size=0.3, random_state=0)

## splitting for generalised_hate
X_train_gh, X_test_gh, y_train_gh, y_test_gh = train_test_split(X,df['generalised_hate'] , test_size=0.3, random_state=0)

## splitting for sarcasm
X_train_sarcasm, X_test_sarcasm, y_train_sarcasm, y_test_sarcasm = train_test_split(X,df['sarcasm'] , test_size=0.3, random_state=0)

## splitting for allegation
X_train_allegation, X_test_allegation, y_train_allegation, y_test_allegation = train_test_split(X,df['allegation'] , test_size=0.3, random_state=0)

## splitting for justification
X_train_justification, X_test_justification, y_train_justification, y_test_justification = train_test_split(X,df['justification'] , test_size=0.3, random_state=0)

## splitting for refutation
X_train_refutation, X_test_refutation, y_train_refutation, y_test_refutation = train_test_split(X,df['refutation'] , test_size=0.3, random_state=0)

## splitting for Support
X_train_Support, X_test_Support, y_train_Support, y_test_Support = train_test_split(X,df['Support'] , test_size=0.3, random_state=0)

## splitting for oppose
X_train_oppose, X_test_oppose, y_train_oppose, y_test_oppose = train_test_split(X,df['oppose'] , test_size=0.3, random_state=0)





## **RANDOM FOREST CLASSIFIER**


In [ ]:
##training for relevance
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train_relevance, y_train_relevance)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
predictions = text_classifier.predict(X_test_relevance)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test_relevance,predictions))    
print(accuracy_score(y_test_relevance, predictions))

[[  88  549]
 [ 224 1613]]
0.6875505254648343


In [ ]:
##training for directed_hate
text_classifier.fit(X_train_dh, y_train_dh)
predictions_dh = text_classifier.predict(X_test_dh)
print(confusion_matrix(y_test_dh,predictions_dh))  
print(accuracy_score(y_test_dh, predictions_dh))






[[2357   11]
 [ 105    1]]
0.9531123686337915


In [ ]:
##training for generalised_hate
text_classifier.fit(X_train_gh, y_train_gh)
predictions_gh = text_classifier.predict(X_test_gh)
print(confusion_matrix(y_test_gh,predictions_gh))    
print(accuracy_score(y_test_gh, predictions_gh))

[[2396   13]
 [  64    1]]
0.9688763136620857


In [ ]:
##training for sarcasm
text_classifier.fit(X_train_sarcasm, y_train_sarcasm)
predictions_sarcasm = text_classifier.predict(X_test_sarcasm)
print(confusion_matrix(y_test_sarcasm,predictions_sarcasm))   
print(accuracy_score(y_test_sarcasm, predictions_sarcasm))

[[2419    6]
 [  48    1]]
0.9781729991915926


In [ ]:
##training for allegation
text_classifier.fit(X_train_allegation, y_train_allegation)
predictions_allegation = text_classifier.predict(X_test_allegation)
print(confusion_matrix(y_test_allegation,predictions_allegation))   
print(accuracy_score(y_test_allegation, predictions_allegation))

[[2307   22]
 [ 141    4]]
0.9341147938561035


In [ ]:
##training for justification
text_classifier.fit(X_train_justification, y_train_justification)
predictions_justification= text_classifier.predict(X_test_justification)
print(confusion_matrix(y_test_justification,predictions_justification))    
print(accuracy_score(y_test_justification, predictions_justification))

[[2389    9]
 [  74    2]]
0.9664510913500404


In [ ]:
##training for refutation
text_classifier.fit(X_train_refutation, y_train_refutation)
predictions_refutation= text_classifier.predict(X_test_refutation)
print(confusion_matrix(y_test_refutation,predictions_refutation))  
print(accuracy_score(y_test_refutation, predictions_refutation))

[[2404    7]
 [  62    1]]
0.9721099434114794


In [ ]:
##training for Support
text_classifier.fit(X_train_Support, y_train_Support)
predictions_Support= text_classifier.predict(X_test_Support)
print(confusion_matrix(y_test_Support,predictions_Support))    
print(accuracy_score(y_test_Support, predictions_Support))

[[1444  213]
 [ 710  107]]
0.6269199676637025


In [ ]:
##training for oppose
text_classifier.fit(X_train_oppose, y_train_oppose)
predictions_oppose= text_classifier.predict(X_test_oppose)
print(confusion_matrix(y_test_oppose,predictions_oppose))   
print(accuracy_score(y_test_oppose, predictions_oppose))

[[2258   33]
 [ 178    5]]
0.9147130153597413


# **NAIVE BAYES**

In [ ]:
##training for relevance
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
NB_model = MultinomialNB()
NB_model.fit(X_train_relevance, y_train_relevance)
y_predict_relevance= NB_model.predict(X_test_relevance)
print(accuracy_score(y_test_relevance, y_predict_relevance))

0.7376717865804365


In [ ]:
##training for directed_hate
NB_model.fit(X_train_dh, y_train_dh)
y_predict_dh= NB_model.predict(X_test_dh)
print(accuracy_score(y_test_dh, y_predict_dh))

0.9563459983831851


In [ ]:
##training for generalised_hate
NB_model.fit(X_train_gh, y_train_gh)
y_predict_gh= NB_model.predict(X_test_gh)
print(accuracy_score(y_test_gh, y_predict_gh))

0.9737267582861763


In [ ]:
##training for sarcasm
NB_model.fit(X_train_sarcasm, y_train_sarcasm)
y_predict_sarcasm= NB_model.predict(X_test_sarcasm)
print(accuracy_score(y_test_sarcasm, y_predict_sarcasm))

0.9801940177849636


In [ ]:
##training for allegation
NB_model.fit(X_train_allegation, y_train_allegation)
y_predict_allegation= NB_model.predict(X_test_allegation)
print(accuracy_score(y_test_allegation, y_predict_allegation))

0.9405820533548909


In [ ]:
##training for justification
NB_model.fit(X_train_justification, y_train_justification)
y_predict_justification= NB_model.predict(X_test_justification)
print(accuracy_score(y_test_justification, y_predict_justification))

0.9692805173807599


In [ ]:
##training for refutation
NB_model.fit(X_train_refutation, y_train_refutation)
y_predict_refutation= NB_model.predict(X_test_refutation)
print(accuracy_score(y_test_refutation, y_predict_refutation))

0.9745351657235246


In [ ]:
##training for Support
NB_model.fit(X_train_Support, y_train_Support)
y_predict_Support= NB_model.predict(X_test_Support)
print(accuracy_score(y_test_Support, y_predict_Support))

0.6608730800323362


In [ ]:
##training for oppose
NB_model.fit(X_train_oppose, y_train_oppose)
y_predict_oppose= NB_model.predict(X_test_oppose)
print(accuracy_score(y_test_oppose, y_predict_oppose))

0.9256265157639451


# **SVC**

In [ ]:
##training for relevance
from sklearn import svm
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_relevance, y_train_relevance)
rbf_pred_relevance = rbf.predict(X_test_relevance)
rbf_accuracy = accuracy_score(y_test_relevance, rbf_pred_relevance)
print(rbf_accuracy)

0.7425222312045271


In [ ]:
##training for directed_hate
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_dh, y_train_dh)
rbf_pred_dh = rbf.predict(X_test_dh)
rbf_accuracy = accuracy_score(y_test_dh, rbf_pred_dh)
print(rbf_accuracy)

0.9571544058205336


In [ ]:
##training for generalised_hate
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_gh, y_train_gh)
rbf_pred_gh = rbf.predict(X_test_gh)
rbf_accuracy = accuracy_score(y_test_gh, rbf_pred_gh)
print(rbf_accuracy)

0.9737267582861763


In [ ]:
##training for sarcasm
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_sarcasm, y_train_sarcasm)
rbf_pred_sarcasm = rbf.predict(X_test_sarcasm)
rbf_accuracy = accuracy_score(y_test_sarcasm, rbf_pred_sarcasm)
print(rbf_accuracy)

0.9801940177849636


In [ ]:
##training for allegation
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_allegation, y_train_allegation)
rbf_pred_allegation = rbf.predict(X_test_allegation)
rbf_accuracy = accuracy_score(y_test_allegation, rbf_pred_allegation)
print(rbf_accuracy)

0.9413904607922393


In [ ]:
##training for justification
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_justification, y_train_justification)
rbf_pred_justification = rbf.predict(X_test_justification)
rbf_accuracy = accuracy_score(y_test_justification, rbf_pred_justification)
print(rbf_accuracy)

0.9692805173807599


In [ ]:
##training for refutation
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_refutation, y_train_refutation)
rbf_pred_refutation = rbf.predict(X_test_refutation)
rbf_accuracy = accuracy_score(y_test_refutation, rbf_pred_refutation)
print(rbf_accuracy)

0.9745351657235246


In [ ]:
##training for Support
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_Support, y_train_Support)
rbf_pred_Support = rbf.predict(X_test_Support)
rbf_accuracy = accuracy_score(y_test_Support, rbf_pred_Support)
print(rbf_accuracy)

0.6697655618431689


In [ ]:
##training for Oppose
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_oppose, y_train_oppose)
rbf_pred_oppose = rbf.predict(X_test_oppose)
rbf_accuracy = accuracy_score(y_test_oppose, rbf_pred_oppose)
print(rbf_accuracy)

0.9260307194826193


# **CNN**

In [ ]:
import numpy as np
import pandas as pd 




from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,CuDNNLSTM,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

df1.head()

ImportError: ignored